[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_case_momentum/EMQA_case_momentum.ipynb)

# EMQA_case_momentum
Momentum trading strategy on Brent crude oil.
**Output:** `case_momentum.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
# Fetch Brent
brent = fetch('BZ=F', start='2018-01-01')
log_ret = np.log(brent / brent.shift(1)).dropna()

# Momentum signal: buy when 20-day return > 0, else cash (0 return)
mom_20 = log_ret.rolling(20).sum()  # 20-day cumulative return
signal = (mom_20 > 0).astype(float).shift(1)  # signal known at end of day, trade next day
signal = signal.loc[log_ret.index].fillna(0)

# Strategy returns
strat_ret = signal * log_ret
buyhold_ret = log_ret

# Cumulative returns
cum_strat = strat_ret.cumsum()
cum_buyhold = buyhold_ret.cumsum()

sharpe_strat = strat_ret.mean() / strat_ret.std() * np.sqrt(252)
sharpe_bh = buyhold_ret.mean() / buyhold_ret.std() * np.sqrt(252)
print(f"Momentum Sharpe:   {sharpe_strat:.3f}")
print(f"Buy-Hold Sharpe:   {sharpe_bh:.3f}")
print(f"Momentum total:    {(np.exp(cum_strat.iloc[-1]) - 1)*100:.1f}%")
print(f"Buy-Hold total:    {(np.exp(cum_buyhold.iloc[-1]) - 1)*100:.1f}%")

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(cum_strat.index, (np.exp(cum_strat) - 1) * 100, color=COLORS['blue'],
        linewidth=1.5, label=f'Momentum (Sharpe={sharpe_strat:.2f})')
ax.plot(cum_buyhold.index, (np.exp(cum_buyhold) - 1) * 100, color=COLORS['gray'],
        linewidth=1.5, label=f'Buy & Hold (Sharpe={sharpe_bh:.2f})')

# Shade momentum-in-market periods
ax.fill_between(signal.index, ax.get_ylim()[0], ax.get_ylim()[1],
                where=signal > 0, alpha=0.05, color=COLORS['green'], label='In Market')

ax.axhline(0, color=COLORS['gray'], linestyle=':', linewidth=0.8)
ax.set_xlabel('Date')
ax.set_ylabel('Cumulative Return (%)')
ax.set_title('Momentum Strategy (20-day) vs Buy & Hold - Brent Crude')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False, ncol=3)

plt.tight_layout()
save_fig(fig, 'case_momentum.pdf')
plt.show()